# Finse meteorological weather stations from the Norwegian Meteorological Service

In [1]:
client_id = 'f07b9306-0fed-4f4f-b4a0-000c4e73f0b5'

In [2]:
import requests 

types='SensorSystem'
# request all stations within a polygon
polygon = 'POLYGON((6.9 60.35,6.9 60.7,8.16 60.7, 8.16 60.35, 6.9 60.35))'
# issue an HTTP GET request
r = requests.get(
        'https://frost.met.no/sources/v0.jsonld',
        {'types': types, 
        'geometry': polygon},
        auth=(client_id, '')
    )



In [3]:
import folium

map = folium.Map(location=[60.5, 7.5], tiles='Stamen Terrain')
map.add_child(folium.LatLngPopup())


for item in r.json()['data']:
        latitute=''
        longitude=''
        county=''
        municipality=''
        if 'geometry' in item:
            latitude = item['geometry']['coordinates'][1]
            longitude = item['geometry']['coordinates'][0]
            location = [latitude,longitude]
        if 'municipality' in item:
            municipality = item['municipality']
        if 'county' in item:
            county = item['county']
        html = """
      <h4>ID: """ + item['id'] + """</h4>
      <h4>Name: """ + item['name'] + """</h4>
      <h4>Latitude: """ + str(latitude) + """</h4>
      <h4>Longitude: """ + str(longitude) + """</h4>
      <h4>Municipality: """ + str(municipality) + """</h4>
      <h4>County: """ + str(county) + """</h4>
      <h4>Country: """ + str(item['country']) + """</h4>
    """ 
        iframe = folium.IFrame(html=html, width=300, height=200)
        popup = folium.Popup(iframe, max_width=2650)
        folium.Marker(location, popup=popup,  icon=folium.Icon(color='blue',  icon='cloud')).add_to(map)



In [4]:
map

In [5]:
map.save('../files/map_metno.html')

In [6]:
help(folium.Icon)

Help on class Icon in module folium.map:

class Icon(branca.element.MacroElement)
 |  Creates an Icon object that will be rendered
 |  using Leaflet.awesome-markers.
 |  
 |  Parameters
 |  ----------
 |  color : str, default 'blue'
 |      The color of the marker. You can use:
 |  
 |          ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
 |           'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
 |           'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
 |           'gray', 'black', 'lightgray']
 |  
 |  icon_color : str, default 'white'
 |      The color of the drawing on the marker. You can use colors above,
 |      or an html color code.
 |  icon : str, default 'info-sign'
 |      The name of the marker sign.
 |      See Font-Awesome website to choose yours.
 |      Warning : depending on the icon you choose you may need to adapt
 |      the `prefix` as well.
 |  angle : int, default 0
 |      The icon will be rotated by this amount of degrees.
 

In [5]:
from pandas.io.json import json_normalize
from datetime import datetime
from beakerx import *

data = r.json()['data']
for element in data:
        value = element['validFrom']
        element['validFrom'] = datetime.strptime(value, "%Y-%M-%d")
MetnoStations = json_normalize(data)
MetnoStations

# Get meteorological data from a given station

- Retrieve observation between start_date and end_date from Finsevatn (SN25830)
- Create a timeseries plot

In [23]:
from pandas.io.json import json_normalize
from datetime import datetime
from beakerx import *
import requests

# example of possible variables to retrieved:
# 'air_temperature', 'surface_air_pressure', 'relative_humidity', 'wind_speed'

referencetime = '2017-04-01/2018-01-01'

# Finsevatn station
source = 'SN25830'
variable = 'air_temperature'
# issue an HTTP GET request
rt = requests.get(
        'https://frost.met.no/observations/v0.jsonld',
        {'sources': source, 'elements': variable,'referencetime': referencetime}, 
        auth=(client_id, '')
)

In [24]:
if rt.status_code == 200:
    # Data available
    T = json_normalize(data=rt.json()['data'], record_path='observations', meta= ['referenceTime', 'sourceId'], errors='ignore')
else:
    print("Data not available")
    

In [25]:
x=T.head(2)

In [26]:
y=x['level'].apply(pd.Series).rename(columns={'unit': 'levelUnit', 'value' : 'levelValue'})

In [27]:
T=pd.concat([T.drop(['level'], axis=1), T['level'].apply(pd.Series).rename(columns={'unit': 'levelUnit', 'value' : 'levelValue'})], axis=1)

In [28]:
T2m = T.loc[T['levelValue'] == 2]

In [29]:
T2m['referenceTime'] = pd.to_datetime(T2m['referenceTime'])
T2m.set_index(['referenceTime'])

/Users/gladys/miniconda3/envs/jupyter_dashboards/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
tsplot = TimePlot(title="2m Temperature from Finse Station SN25830",
     xLabel="Date",
     yLabel="2m temperature (degrees Celcius)" , 
     crosshair = Crosshair(color=Color.gray, width= 2, style= StrokeType.DOT))
tsplot.add(Line(x=T2m['referenceTime'], y=T2m['value'], color=Color.green, displayName="T2m") )